# Лабораторная работа №2

## Задания

Решить задачу классификации датасета MNIST используя MLP из scikitlearn и используя CNN (по типу LeNet) c пакетом PyTorch. Сравнить результаты по метрикам, сделать обоснованные выводы

## Реализация

### Библиотеки

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

### Датасет

In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
y = y.astype(np.int64)  # т.к. там классификация цифр, то лучше если брать по числовому значению
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)  # stratify нужен, чтобы все классы были так или иначе включены

In [3]:
def one_hot_transform(y, classes_count):  # преобразование в матрицу вхождений
    transformed = np.zeros((len(y), classes_count))

    for i in range(len(y)):  # исходит из предположения, что классы будут пронумерованы по порядку, начиная от 0
        transformed[i][y[i]] = 1

    return transformed

In [4]:
y_test_proba = one_hot_transform(y_test, 10)

### Multi-layer Perceptron classifier

In [5]:
clf = MLPClassifier(early_stopping=True).fit(X_train, y_train)  # early stopping чтобы не ждать слишком долго

In [6]:
y_mlp_proba = clf.predict_proba(X_test)
y_mlp = clf.predict(X_test)

### LeNet (Convolutional Neural Network)

LeNet состоит из следующих слоёв:

1. Входной слой: изображение 28x28 пикселей, 1 канал
2. Свёрточный слой: ядро 5x5, padding 2, 6 каналов на выходе, функция активации - гиперболический тангенс
3. AvgPooling: ядро 2x2, stride 2
4. Свёрточный слой: ядро 5x5, 16 каналов на выходе, функция активации - гиперболический тангенс
5. AvgPooling: ядро 2x2, stride 2
6. Уплотнение
7. Плотный слой из 120 нейронов, функция активации - гиперболический тангенс
8. Плотный слой из 84 нейронов, функция активации - гиперболический тангенс
9. Выходной (плотный) слой из 10 нейронов (т.к. цифры от 0 до 9)

Padding нужен для того, чтобы улавливать более тонкие детали - вроде углов картинок. Если бы входное изображение было размером 32x32, то тогда в padding не было бы нужды

In [7]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Свёрточный слой 1
        # Было изображение 28x28x1, стало 28x28x6, размер не уменьшился т.к. padding добавил к каждой стороне 2

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2) 

        # Свёрточный слой 2
        # Был mapping 14x14x6, стал 10x10x16, как видно, размер уменьшился

        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        # AvgPooling
        # Уменьшает размер в 2 раза, т.к. stride = 2 (stride = размерность "куска" для пулинга)
        # Пример: картинку 10x10 просканировали квадратом 2x2 и взяли среднее в этом квадрате, на выходе получится картинка 5x5)

        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        # Плотный слой 1
        # 400 т.к. после pooling будут мэппинги размером 5x5 и их будет 16 штук, т.е. 5*5*16=400
        
        self.dense1 = nn.Linear(in_features=400, out_features=120) 

        # Плотный слой 2

        self.dense2 = nn.Linear(in_features=120, out_features=84)

        # Плотный слой 3 (выходной)

        self.dense3 = nn.Linear(in_features=84, out_features=10)

        # Функция активации

        self.activation = nn.Tanh()


    def forward(self, x):  # теперь определяем порядок выполнения
        out = self.conv1(x)
        out = self.activation(out)
        
        out = self.pool(out)
        
        out = self.conv2(out)
        out = self.activation(out)
        
        out = self.pool(out)
        out = torch.flatten(out, start_dim=1)  # уплотняем
        
        out = self.dense1(out)
        out = self.activation(out)
        
        out = self.dense2(out)
        out = self.activation(out)
        
        out = self.dense3(out)

        return out

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # чтобы быстрее обучалось, использую cuda
lenet = LeNet().to(device)

### Нормализация данных

Необходимо для корректного обучения

In [9]:
mean = 0.1307  # уже просчитанные значения для этого датасета
std = 0.3081

images_train = (X_train/255 - mean)/std
images_test = (X_test/255 - mean)/std

images_train = images_train.reshape(-1, 1, 28, 28)
images_test = images_test.reshape(-1, 1, 28, 28)

### Преобразование датасета

PyTorch требует, чтобы датасет был определённого формата

In [10]:
train_dataset = TensorDataset(torch.tensor(images_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(images_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Обучение LeNet

Теперь определяем, как будет обучаться модель. Для этого нужна функция ошибок и оптимизатор (Adam или градиентный спуск)

In [11]:
cost = nn.CrossEntropyLoss()
optimiser = optim.Adam(lenet.parameters(), lr=10**-3)

In [12]:
total_epochs = 10

for epoch in range(total_epochs):
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = lenet(images)
        loss = cost(outputs, labels)
        
        optimiser.zero_grad()  # обнулить градиенты
        loss.backward()
        optimiser.step()  # провести оптимизацию

        running_loss += loss.item()

    print(f"Epoch {epoch+1} out of {total_epochs}, loss is {running_loss/len(train_loader)}")  # делим,т.к. running_loss - это сумма ошибки каждого батча, а не модели

Epoch 1 out of 10, loss is 0.3550818792913662
Epoch 2 out of 10, loss is 0.09999014626182344
Epoch 3 out of 10, loss is 0.0647959651679272
Epoch 4 out of 10, loss is 0.0491717707710182
Epoch 5 out of 10, loss is 0.037563357588329745
Epoch 6 out of 10, loss is 0.03041205348825386
Epoch 7 out of 10, loss is 0.02415721211922321
Epoch 8 out of 10, loss is 0.020599473506289278
Epoch 9 out of 10, loss is 0.01733624787634089
Epoch 10 out of 10, loss is 0.01633281289662717


### Предсказание LeNet

In [13]:
lenet.eval()  # переводим в режим оценки

y_net_proba = []
y_net =[]

with torch.no_grad():  # уменьшить потребление памяти
    for images, labels in test_loader:
        images = images.to(device)
        outputs = lenet(images)

        probs = torch.softmax(outputs, dim=1)  # вероятности

        for proba in probs.cpu().numpy():  
            y_net_proba.append(proba)

        for label in outputs.argmax(dim=1).cpu().numpy():  # предсказанные классы
            y_net.append(label)

y_net_proba = np.array(y_net_proba)
y_net = np.array(y_net)

### Метрики MLP

In [14]:
accuracy_score(y_test, y_mlp)

0.964

In [15]:
recall_score(y_test, y_mlp, average="macro")

0.9637370570049193

In [16]:
precision_score(y_test, y_mlp, average="macro")

0.9638747294852854

In [17]:
f1_score(y_test, y_mlp, average="macro")

0.9636953445317215

In [18]:
roc_auc_score(y_test_proba, y_mlp_proba)

0.9980145358515617

### Метрики LeNet

In [19]:
accuracy_score(y_test, y_net)

0.9864

In [20]:
recall_score(y_test, y_net, average="macro")

0.9863082778861536

In [21]:
precision_score(y_test, y_net, average="macro")

0.9863085085595529

In [22]:
f1_score(y_test, y_net, average="macro")

0.9862861736112171

In [23]:
roc_auc_score(y_test_proba, y_net_proba)

0.9997987418015242

### Вывод

По результатам метрик можно сделать вывод, что LeNet лучше может определить, к какому типу относится картинка

Обе модели хорошо умеют отличать классы друг от друга, но LeNet всё же точнее MLP